## 4.0 Capstone Project Two: All State Purchase Prediction - Pre Processing and Training Data DEvelopment<a id='2_Exploratory_Data_Analysis'></a>
**Submitted By:** Amit Kukreja

## Objectives<a id='2.2_EDA_Objectives'></a>

1) Convert the dataset from long form to wide form.

2) Split the data into training and test set into separate csv files and only use training data going forward.

3) Make a data frame with one row per customer in the training set, that includes the data from the first two shopping points, and then the final shopping point as the target.

4) Try to write a function (or a class even better) that can take in any number of shopping points and make a dataframe with that number of shopping points 

5) Create dummy or indicator features for categorical variables.

6) Standardize the magnitude of numeric features using a scaler. 

7) Create a class that performs the pre processing steps and stores the dataframes and scaler as attributes.




In [1]:
from pathlib import Path

import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from pandas_profiling.utils.cache import cache_file
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sb_utils import save_file
from collections import defaultdict
from collections import Counter
from sklearn.model_selection import train_test_split


## How to use this notebook for pre-processing a new dataset

Follow along the notebook for section headers marked step 1, step 2 etc. These sections perform the steps explained below.
It's a good practise to save output of a step into a csv file before moving onto the next step. If for any reason you make an error in a step, you can reload the output of the previous step from the saved file.

**Step 1: Convert the dataset from long form to wide form using the expand horizontal function.**

**Step 2 (optional): Split the wide data into training and test sets. If making new prediction, skip this step.**

**Step 3 - Extract the required quote history from the dataset** 
* Create an instance of the QuoteHistory() class and pass the wide form dataframe to it using the ‘pass_data’ method. 
* Then use the ‘get_history’ method, setting argument how=‘first2’ to get columns related to the first two quotes. 
* To get a different set of quotes, follow steps in the sub-section 'Extracting different set of quote history'.

**Step 4 - Pre-process and Transform data** 
* Create an instance of the PreProcess() class and pass the dataframe from step 3 to it using the ‘dataset’ method. 
* Then use the ‘transform’ method, setting argument quote_nos=‘[1,2]’ to get X and y dataframes to be used for modeling. The list passed on through the 'quote_nos' argument must contain the quote nos. extracted in step 3, so if a different set of quote nos. has been extracted, then pass the relevant quote nos. 
* The 'transform' method returns X and y dataframes. X contains the preprocessed and transformed feature columns and y contains the 7 target vectors under columns ‘A’ through ‘G’. Save X and y dataframes in separate csv files.
* The steps are slightly different for train and test sets, so follow along instructions given in the section.




## Step 1: Convert the dataset from long form to wide form

In [44]:
#Let's write a function to expand the dataset horizontally so that there is one row per customer 
# and the customer's entire shopping history is captured in that row
# The function takes 2 argument
#  1) df_long: The long form dataset
#  2) dataset_type: pass 'train' if expanding a training dataset that includes the final purchase of ALL customers 
#                  pass 'test' if dataset has partial history(NO final purchase) of customers and predictions have to  
#                   be made on their final vector choices. 

def expand_horizontal(df_long, dataset_type = 'train'):
    """ function to expand a long form dataset to wide from"""
    
    quote_info = ['customer_ID', 'group_size','homeowner','car_age','car_value','risk_factor',\
                'age_oldest','age_youngest', 'married_couple','C_previous','duration_previous', \
                'A', 'B', 'C', 'D', 'E', 'F', 'G','cost']
    
    max_shopping_pt = df_long['shopping_pt'].max()
    
    if dataset_type == 'train':
        df_wide = df_long[df_long['record_type']==1]

        for quote in range(1,max_shopping_pt+1):
            df_temp = df_long[df_long['shopping_pt']==quote][quote_info]
            df_wide = df_wide.merge(df_temp, on = 'customer_ID', how='left', suffixes=["", "_"+str(quote)])
            
    elif dataset_type == 'test':
        df_wide = df_long[df_long['shopping_pt']==1]
        
        for quote in range(2, max_shopping_pt+1):
            df_temp = df_long[df_long['shopping_pt']==quote][quote_info]
            df_wide = df_wide.merge(df_temp, on = 'customer_ID', how='left', suffixes=["", "_"+str(quote)])
           
        df_wide.rename(columns={'A':'A_1', 'B':'B_1', 'C':'C_1', 'D':'D_1', 'E':'E_1', \
                                'F':'F_1', 'G':'G_1', 'cost': 'cost_1'}, inplace=True)
        
    
    return df_wide


In [45]:
# Load the dataset 

df_train = pd.read_csv('WIP_data/data_with_filled_missing_vals.csv')
df_train.shape

(665249, 25)

In [46]:
# Call the expand_horizontal function with 'dataset_type' set appropriately (see comments above function definition)

df_wide = expand_horizontal(df_train, dataset_type = 'train')
                            # set dataset_type = 'test' if expanding a dataset without final purchase of customers

df_wide.shape


(97009, 259)

In [47]:
# save the wide form dataset in a csv file

save_file(df_wide, 'training_dataset_wide_form.csv', 'WIP_data')


A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "WIP_data\training_dataset_wide_form.csv"


## Step 2 (optional): Split the dataset into training and validation set 

In [30]:
# Let's read the wide dataframe saved to file in Step 1 into a dataframe object

df_wide = pd.read_csv("WIP_data/training_dataset_wide_form.csv")

df_wide.head()

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,...,C_previous_13,duration_previous_13,A_13,B_13,C_13,D_13,E_13,F_13,G_13,cost_13
0,10000000,9,1,0,12:07,IN,10001,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000005,6,1,3,09:09,NY,10006,1,0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000007,8,1,4,14:26,PA,10008,1,0,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000013,4,1,4,09:31,WV,10014,2,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000014,6,1,1,17:50,MO,10015,1,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Split the dataframe into training and validation(test) sets with test being 20% of the observations
# We use stratify on the shopping point column so that both training and test sets contain a similar proportion of all 
# shopping points. Straify is important here as for some shopping points e.g. 12 & 13, very few datapoints are available.

df_train, df_test = train_test_split(df_wide, test_size = 0.2, random_state = 123, stratify = df_wide['shopping_pt'])


In [32]:
df_train.shape, df_test.shape

((77607, 259), (19402, 259))

In [64]:
# save the train and test sets in separate files.

datapath = "WIP_data"

save_file(df_train, 'training_data.csv', datapath)
save_file(df_test, 'test_data.csv', datapath)


A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "WIP_data\training_data.csv"
A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "WIP_data\test_data.csv"


## Step 3 - Extract the required quote history from the dataset

In [2]:
# Now we load only the training data into a dataframe

df_wide = pd.read_csv("WIP_data/training_data.csv")

df_wide.head()

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,...,C_previous_13,duration_previous_13,A_13,B_13,C_13,D_13,E_13,F_13,G_13,cost_13
0,10109793,9,1,4,14:44,CO,13320,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10002231,8,1,5,13:36,OH,10601,1,0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10150024,8,1,4,15:33,OH,10081,3,1,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10003949,8,1,4,09:13,FL,10302,2,1,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10103809,10,1,2,11:06,FL,14844,1,1,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_wide.shape

(77607, 259)

In [4]:
# The training data has the entire shopping history of the customer.
# we want to predict the final product vectors (target) based on early shopping history of the customer
# So we define a class that contains the entire shopping history and has an in-built function to extract 
# any number of shopping points as we want

class QuoteHistory:
    #class to contain customer data and extract appropriate quote history
    
    def pass_data(self, dataframe):
        # initialize the shopping_history object with customer data across all shopping points
        self.data = dataframe
        
    def get_history(self, dataset_type = 'train', how = 'first2', quote_nos=[]):
        
        customer_data = ['group_size', 'homeowner', 'car_age', 'car_value', 'risk_factor', 'age_oldest', 'age_youngest', \
                 'married_couple', 'C_previous', 'duration_previous']
        product_vectors = ['A', 'B', 'C','D','E','F','G', 'cost']

        def hist_extract(quotes=[1,2]):
            
            if dataset_type == 'train':
                df_temp = self.data[self.data['shopping_pt'] > np.max(quotes)]\
                                  [['customer_ID','shopping_pt','state', 'A', 'B', 'C','D','E','F','G', 'cost']]
            
            elif dataset_type == 'test':
                df_temp = self.data[['customer_ID','state']]
            
            vector_cols = ['customer_ID']+[x+'_'+str(y) for x in product_vectors for y in quotes]
            customer_data_cols = ['customer_ID']+[x+'_'+str(np.max(quotes)) for x in customer_data]

            df_temp = df_temp.merge(self.data[customer_data_cols], on='customer_ID', how='left', suffixes=["",""])
            df_temp = df_temp.merge(self.data[vector_cols], on='customer_ID', how='left', suffixes=["",""])
            
            return df_temp

  
        if how == 'first2':
                                
            return hist_extract([1,2])
        
        elif how == 'first3':
            return hist_extract([1,2,3])
        
        elif how == 'specific' and quote_nos != []:
            
            return hist_extract(quote_nos)
        
        elif how == 'last2':
                
            df_temp = self.data.iloc[:, 0:25]
            
            for index, row in self.data.iterrows():
                print(index)
                quote_second_last = self.data.loc[index, 'shopping_pt'] - 2
                quote_last = self.data.loc[index, 'shopping_pt'] - 1

                for feature in quote_features:
                    df_temp.loc[index, feature+'_2nd_last'] = self.data.loc[index, feature+"_q_"+str(quote_second_last)]
                    df_temp.loc[index, feature+'_last'] = self.data.loc[index, feature+"_q_"+str(quote_last)]
            
            return df_temp



In [5]:
# Create an instance of the QuoteHistory() class and pass the wide form dataframe to it using the ‘pass_data’ method. 
# Then use the ‘get_history’ method, setting argument how=‘first2’ to get columns related to the first two quotes. 
# To get a different set of quotes, follow steps in the sub-section 'Extracting different set of quote history'.

df_2 = QuoteHistory()
df_2.pass_data(df_wide)

df_hist_first2 = df_2.get_history(dataset_type = 'train', how='first2')
df_hist_first2.head()


,customer_ID,shopping_pt,state,A,B,C,D,E,F,G,...,D_1,D_2,E_1,E_2,F_1,F_2,G_1,G_2,cost_1,cost_2
0,10109793,9,CO,1,1,3,3,0,2,1,...,3,3,0,0,2,2,1,1,656,656
1,10002231,8,OH,0,0,1,3,0,0,3,...,1,3,0,0,2,0,3,3,598,557
2,10150024,8,OH,1,1,2,3,0,2,3,...,3,3,0,0,3,3,3,3,617,617
3,10003949,8,FL,1,1,2,2,1,2,3,...,2,2,1,1,2,2,4,3,647,675
4,10103809,10,FL,1,1,1,3,1,1,3,...,3,3,1,1,1,1,4,3,637,617


In [17]:
datapath = "WIP_data"

save_file(df_hist_first2, 'training_data_with_first2_quotes.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "WIP_data\training_data_with_first2_quotes.csv"


### Step 3: Extracting different set of quote history (other than first 2 quotes)

To extract a different set of quotes, follow the following steps:

* Create an instance of QuoteHistory()

* pass the wide form dataset to it using 'pass_data" method

* Use the 'get_history' method with following arguments
           - dataset_type = 'train' or 'test' as required
           - how = 'specific'
           - quote_nos = [provide a list of quote nos. to be extracted separated by commas]
               

In [31]:

df_wide = pd.read_csv("WIP_data/test_dataset_wide_form_ver2.csv")
df_wide.shape     

C:\Users\amite\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (190) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(55716, 205)

In [10]:


df_2 = QuoteHistory()
df_2.pass_data(df_wide)

df_hist_1_2_3 = df_2.get_history(dataset_type = 'test', how='specific', quote_nos=[1,2,3])

df_hist_1_2_3.head()


,customer_ID,state,group_size_3,homeowner_3,car_age_3,car_value_3,risk_factor_3,age_oldest_3,age_youngest_3,married_couple_3,...,E_3,F_1,F_2,F_3,G_1,G_2,G_3,cost_1,cost_2,cost_3
0,10000001,OK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,3,NaN,4,2,NaN,543,611,NaN
1,10000002,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,2,NaN,2,2,NaN,691,695,NaN
2,10000003,AR,1.0,0.0,4.0,d,4.0,26.0,26.0,0.0,...,0.0,2,2,2.0,2,2,2.0,628,625,628.0
3,10000004,OK,1.0,0.0,13.0,f,3.0,22.0,22.0,0.0,...,0.0,0,3,3.0,2,2,2.0,596,711,722.0
4,10000006,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,NaN,1,1,NaN,625,625,NaN


In [11]:
df_hist_1_2_3.columns

Index(['customer_ID', 'state', 'group_size_3', 'homeowner_3', 'car_age_3',
       'car_value_3', 'risk_factor_3', 'age_oldest_3', 'age_youngest_3',
       'married_couple_3', 'C_previous_3', 'duration_previous_3', 'A_1', 'A_2',
       'A_3', 'B_1', 'B_2', 'B_3', 'C_1', 'C_2', 'C_3', 'D_1', 'D_2', 'D_3',
       'E_1', 'E_2', 'E_3', 'F_1', 'F_2', 'F_3', 'G_1', 'G_2', 'G_3', 'cost_1',
       'cost_2', 'cost_3'],
      dtype='object')

In [12]:
datapath = "WIP_data"

save_file(df_hist_1_2_3, 'test_data_with_quotes_1_2_3.csv', datapath)

Writing file.  "WIP_data\test_data_with_quotes_1_2_3.csv"


## Step 4 - Pre-process and Transform data


In [13]:
df_train_first2 = pd.read_csv('WIP_data/training_data_with_first2_quotes.csv')
df_train_first2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77607 entries, 0 to 77606
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_ID          77607 non-null  int64  
 1   shopping_pt          77607 non-null  int64  
 2   state                77607 non-null  object 
 3   A                    77607 non-null  int64  
 4   B                    77607 non-null  int64  
 5   C                    77607 non-null  int64  
 6   D                    77607 non-null  int64  
 7   E                    77607 non-null  int64  
 8   F                    77607 non-null  int64  
 9   G                    77607 non-null  int64  
 10  cost                 77607 non-null  int64  
 11  group_size_2         77607 non-null  int64  
 12  homeowner_2          77607 non-null  int64  
 13  car_age_2            77607 non-null  int64  
 14  car_value_2          77607 non-null  int64  
 15  risk_factor_2        77607 non-null 

In [22]:
df_train_first2.head()

,customer_ID,shopping_pt,state,A,B,C,D,E,F,G,...,D_1,D_2,E_1,E_2,F_1,F_2,G_1,G_2,cost_1,cost_2
0,10109793,9,CO,1,1,3,3,0,2,1,...,3,3,0,0,2,2,1,1,656,656
1,10002231,8,OH,0,0,1,3,0,0,3,...,1,3,0,0,2,0,3,3,598,557
2,10150024,8,OH,1,1,2,3,0,2,3,...,3,3,0,0,3,3,3,3,617,617
3,10003949,8,FL,1,1,2,2,1,2,3,...,2,2,1,1,2,2,4,3,647,675
4,10103809,10,FL,1,1,1,3,1,1,3,...,3,3,1,1,1,1,4,3,637,617


We see that all the product vector columns are stored as integers. These features are categorical as customer is choosing discrete options for the product vector. So they must be changed to categorical i.e. object.
Similarly, there are certain customer info features that have discrete values but are stored as integers. These are:

 - group_size_2: Takes values (1,2,3,4)           
 
 - homeowner_2: Takes values (0.1)
 
 - car_value_2: Takes values (1 to 9)
 
 - risk_factor_2: Takes values (0,1,2,3,4)
 
 - married_couple_2: Takes values (0,1)
 
 - C_previous_2: Takes values (1,2,3,4)
 
 
 
The above features along with product vectors need to be converted to categorical.
 
The features that take continuous values are cost, car_age_2, age_oldest_2, age_youngest_2, and duration_previous_2. Only duration previous is stored as float, the rest are stored as integers. We shall convert the others to float as well.
 
While 'shopping_pt' takes values 1 to 13, we shall keep it as as numerical variable as there is a 'distance' quality in this feature, customers with larger shopping points travel more 'distance' in their purchase process.

In [19]:
# To pre-process our dataset, we define a class. This class will hold the dataset and perform all the operations necessary
# to make the dataset ready for modeling stage.

class PreProcess:
    #class to PreProcess the dataset created in Step 3 and make it ready for modeling
    
    def dataset(self, dataframe):
        # method to pass dataset to the class objecy
        
        self.data = dataframe
        
    def transform(self, quote_nos=[1,2], dataset_type = 'train', scaler = StandardScaler()):
        
        # To transform the dataframe and make it ready for modeling/predictions, we need to:
        # 1) add new columns a) No. of vector changes made at each quote b) cost difference between each set of quotes
        # 2) scale the numeric features
        # 3) dummy encode the categorical features
        # 4) Encode vectors C, D & G from 0 to 3 and vector D as (0,1,2) as XG boost doesn't accept encoding starting at 1.
        
        # Let's define a function to count changes made by each customer for given dataframe, vectors and shopping points
        # returns a dataframe with customer_IDs and corresponding changes at each step

        def changes_by_shopping_pt(dataframe, product_vectors, shp_pt):
    
            cust_tracker = dataframe[['customer_ID']]

            for idx, quote in enumerate(shp_pt):
                if idx >= 1:
        
                    cols_to_select = ['customer_ID']
                    for vector in product_vectors:
                        cols_to_select.append(vector+"_"+str(shp_pt[idx-1]))
                        cols_to_select.append(vector+"_"+str(shp_pt[idx]))
        
                    cols_to_compare = dataframe[dataframe['shopping_pt'] >= quote][cols_to_select]
                    cols_to_sum = []

                    for vector in product_vectors:
    
                        # count how many customers changed vector from previous quote to current quote
                        cols_to_compare[str(quote)+vector] = np.where(cols_to_compare[vector+"_"+str(shp_pt[idx])] \
                                                       != cols_to_compare[vector+"_"+str(shp_pt[idx-1])], 1, 0)
        
                        cols_to_sum.append(str(quote)+vector)
        
                    cols_to_compare['changes_step_'+str(quote)] = cols_to_compare[cols_to_sum].sum(axis=1)
    
                    cust_tracker = cust_tracker.merge(cols_to_compare[['customer_ID', 'changes_step_'+str(quote)]], on = 'customer_ID', how='left')
        
            return cust_tracker

        def change_vector_encoding(quote_nos):
            change_vectors = ['C', 'D', 'G']
            new_encoding = {1:0, 2:1, 3:2, 4:3}
            for vect in change_vectors:
                self.data[vect] = self.data[vect].map(lambda x: new_encoding[x])
                for quote in quote_nos:
                    self.data[vect+'_'+str(quote)] = self.data[vect+'_'+str(quote)].map(lambda x: new_encoding[x])
            
            return self.data
            
        self.data = change_vector_encoding(quote_nos)
        
        self.data = self.data.merge(
            changes_by_shopping_pt(self.data, ['A', 'B', 'C','D','E','F','G'], quote_nos), on='customer_ID', how='left')
        
        for idx, quote in enumerate(quote_nos):
            if idx >= 1:
                self.data['cost_diff_step_'+str(quote)] = self.data['cost_'+str(quote_nos[idx])] - self.data['cost_'+str(quote_nos[idx-1])]
        
        num_features = ['shopping_pt'] + ['cost_'+str(quote) for quote in quote_nos] \
                        +['car_age_'+str(quote_nos[-1])]+ ['age_oldest_'+str(quote_nos[-1])] \
                        + ['age_youngest_'+str(quote_nos[-1])] + ['duration_previous_'+str(quote_nos[-1])] \
                        + ['cost_diff_step_'+str(quote) for idx, quote in enumerate(quote_nos) if idx >= 1] \
                        + ['changes_step_'+str(quote) for quote in quote_nos[1:]]
        
        catg_features = [x for x in self.data.columns if x not in num_features]

        self.data[catg_features] = self.data[catg_features].astype('object')
        self.data[num_features] = self.data[num_features].astype('float')
        
        # From self.data, we need to drop the customer_ID and target(final product vector) columns to create 
        # our features dataset X. Also, we shall define y with the target vectors and policy cost.
        
        X = self.data.drop(columns=['customer_ID', 'A','B','C','D','E','F','G','cost'])
        y = self.data[['A','B','C','D','E','F','G']]
        
        # Next, we do dummy encoding for the categorical columns and scale the numeric columns
        
        X = pd.get_dummies(X, drop_first=True)
        
        # We store the scaler as an attribute of our class
        if dataset_type == 'train':
            self.scaler = StandardScaler().fit(X[num_features])
        elif dataset_type == 'test':
            self.scaler = scaler
        
        X[num_features] = self.scaler.transform(X[num_features])
        
        # We split the dataset into training and test sets and store them as attribute as well
        
        return X, y
        
        


### Step 4 (Type 1) - Transforming a training dataset

**Important** : When we are transforming a training dataset, we want to save the PreProcess object into a pickle file, so that its 'scaler' attribute can be used later on for the test/validation datasets.  

**Steps**
* Create an instance of the PreProcess() class and pass the dataframe from step 3 to it using the ‘dataset’ method.
* Then use the ‘transform’ method, setting arguments as follows:
           - dataset_type = 'train'
           - quote_nos = ‘[1,2]’ The list passed on through the 'quote_nos' argument must contain the quote nos. extracted in step 3, so if a different set of quote nos. has been extracted, then pass the relevant quote nos.
            
* The 'transform' method returns X and y dataframes. X contains the preprocessed and transformed feature columns and y contains the 7 target vectors under columns ‘A’ through ‘G’. Save X and y dataframes in separate csv files.
* The 'transform' method also creates a 'scaler' attribute for the object. So the PreProcess object can be stored in a pickle file as its 'scaler' attribute would be required later to transform the numeric features of a test dataset.

In [ ]:
pp = PreProcess()
pp.dataset(df_train_first2)

X, y = pp.transform(dataset_type = 'train', quote_nos=[1,2])

X.shape, y.shape

In [21]:
X.head(7)

,shopping_pt,car_age_2,age_oldest_2,age_youngest_2,duration_previous_2,cost_1,cost_2,changes_step_2,cost_diff_step_2,state_AR,...,F_1_3,F_2_1,F_2_2,F_2_3,G_1_1,G_1_2,G_1_3,G_2_1,G_2_2,G_2_3
0,1.072089,-0.901461,-0.003780,0.129103,-0.636138,0.464913,0.467906,-0.871664,-0.043014,0,...,0,0,1,0,0,0,0,0,0,0
1,0.571683,0.824464,-1.209101,-1.073002,0.862673,-0.677346,-1.614463,0.903450,-1.338349,0,...,0,0,0,0,0,1,0,0,1,0
2,0.571683,2.032611,-0.061176,-1.359217,-0.636138,-0.303158,-0.352421,-0.871664,-0.043014,0,...,1,0,0,1,0,1,0,0,1,0
3,0.571683,-0.211091,-0.692535,-0.615057,-0.636138,0.287666,0.867553,-0.279959,0.841605,0,...,0,0,1,0,0,0,1,0,1,0
4,1.572496,0.134094,-1.036912,-0.901272,-0.422023,0.090725,-0.352421,0.311745,-0.674885,0,...,0,1,0,0,0,0,1,0,1,0
5,-1.930351,-0.211091,1.718107,1.846396,1.290905,-1.760522,-0.015876,3.270269,2.800404,0,...,0,1,0,0,0,0,1,1,0,0
6,0.071276,-0.211091,1.086749,1.216722,-0.850254,0.701243,0.594111,-0.279959,-0.232575,0,...,0,0,1,0,0,1,0,0,1,0


In [22]:
y.head()

,A,B,C,D,E,F,G
0,1,1,2,2,0,2,0
1,0,0,0,2,0,0,2
2,1,1,1,2,0,2,2
3,1,1,1,1,1,2,2
4,1,1,0,2,1,1,2


In [24]:
pp.scaler

StandardScaler()

In [12]:
datapath2 = "Transformed_data"

save_file(X, 'pre_processed_training_data_with_quotes_1_2.csv', datapath2)
save_file(y, 'training_data_target_columns.csv', datapath2)


A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "Transformed_data\pre_processed_training_data_with_quotes_1_2.csv"
A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "Transformed_data\training_data_target_columns.csv"


In [14]:
# Let's also save the pre_processor object

save_file(pp, 'pre_processor.pkl', datapath2)


Writing file.  "Transformed_data\pre_processor.pkl"


### Step 4 (Type 2) - Transforming a validation/test dataset

**Important** : When we are transforming a validation dataset, we want to retreive the PreProcess object from Step 4(Type 1) that was saved to a pickle file and use its 'scaler' attribute for transforming the numeric features of the validation dataset.

**Steps**

* Create an instance of the PreProcess() class and pass the validation/test dataframe from step 3 to it using the ‘dataset’ method.
* Then use the ‘transform’ method, setting arguments as follows:
         - dataset_type = 'test'
         - quote_nos = ‘[1,2]’ The list passed on through the 'quote_nos' argument must contain the quote nos. extracted in step 3, so if a different set of quote nos. has been extracted, then pass the relevant quote nos.
         - scaler = use scaler attribute of the PreProcess object from Step 4(Type 1) that was saved to a pickle file
     
* The 'transform' method returns X and y dataframes. X contains the preprocessed and transformed feature columns and y contains the 7 target vectors under columns ‘A’ through ‘G’. Save X and y dataframes in separate csv files.


In [25]:
# Load the file with test data (first two quotes extracted in step 3)

df_test_first2 = pd.read_csv('Test_data/test_data_with_first2_quotes.csv')

df_test_first2.head()


,customer_ID,shopping_pt,state,A,B,C,D,E,F,G,...,D_1,D_2,E_1,E_2,F_1,F_2,G_1,G_2,cost_1,cost_2
0,10129104,5,FL,1,1,3,3,0,1,3,...,3,3,0,0,1,1,3,3,606,606
1,10095371,7,WV,2,0,1,1,0,2,1,...,1,2,0,0,0,2,4,1,596,675
2,10032679,8,MO,2,1,3,3,1,2,3,...,3,3,1,1,1,2,3,3,614,617
3,10025603,5,NY,0,1,4,3,1,0,2,...,3,3,1,1,0,0,2,2,599,590
4,10122799,8,NY,2,1,3,3,1,0,2,...,3,3,1,1,0,0,4,2,724,717


In [26]:
df_test_first2.shape

(19402, 37)

In [27]:
# Load the pre-processor training data object from Step 4(Type 1) that was saved to a pickle file
import pickle

with open('Transformed_data/pre_processor.pkl', 'rb') as f:
    pp_train_first2 = pickle.load(f)

type(pp_train_first2)


__main__.PreProcess

In [28]:
# Pre-process test data, but use the scaler from training data

pp_test = PreProcess()
pp_test.dataset(df_test_first2)

X, y = pp_test.transform(quote_nos=[1,2], dataset_type = 'test', scaler = pp_train_first2.scaler)

X.shape, y.shape

((19402, 94), (19402, 7))

In [33]:
X.head()

,shopping_pt,car_age_2,age_oldest_2,age_youngest_2,duration_previous_2,cost_1,cost_2,changes_step_2,cost_diff_step_2,state_AR,...,F_1_3,F_2_1,F_2_2,F_2_3,G_1_1,G_1_2,G_1_3,G_2_1,G_2_2,G_2_3
0,-0.929538,0.306686,1.718107,1.331208,-1.064370,-0.519793,-0.583795,-0.871664,-0.043014,0,...,0,1,0,0,0,1,0,0,1,0
1,0.071276,-0.901461,-1.151705,-1.015758,-1.064370,-0.716734,0.867553,2.086860,2.452875,0,...,0,0,1,0,0,0,1,0,0,0
2,0.571683,-0.211091,0.742371,0.587048,-1.064370,-0.362240,-0.352421,-0.279959,0.051766,0,...,0,0,1,0,0,1,0,0,1,0
3,-0.929538,-1.246646,-0.061176,0.071860,1.933252,-0.657652,-0.920340,-0.871664,-0.327356,0,...,0,0,0,0,1,0,0,1,0,0
4,0.571683,-1.246646,-1.094309,-0.958515,-0.422023,1.804113,1.750982,-0.279959,-0.264169,0,...,0,0,0,0,0,0,1,1,0,0


In [34]:
y.head()

,A,B,C,D,E,F,G
0,1,1,2,2,0,1,2
1,2,0,0,0,0,2,0
2,2,1,2,2,1,2,2
3,0,1,3,2,1,0,1
4,2,1,2,2,1,0,1


In [35]:
datapath = "Test_data"

save_file(X, 'pre_processed_test_data_with_quotes_1_2.csv', datapath)
save_file(y, 'test_data_target_columns.csv', datapath)


Writing file.  "Test_data\pre_processed_test_data_with_quotes_1_2.csv"
Writing file.  "Test_data\test_data_target_columns.csv"


In [30]:
pp_test.data.head()

,customer_ID,shopping_pt,state,A,B,C,D,E,F,G,...,E_1,E_2,F_1,F_2,G_1,G_2,cost_1,cost_2,changes_step_2,cost_diff_step_2
0,10129104,5.0,FL,1,1,2,2,0,1,2,...,0,0,1,1,2,2,606.0,606.0,0.0,0.0
1,10095371,7.0,WV,2,0,0,0,0,2,0,...,0,0,0,2,3,0,596.0,675.0,5.0,79.0
2,10032679,8.0,MO,2,1,2,2,1,2,2,...,1,1,1,2,2,2,614.0,617.0,1.0,3.0
3,10025603,5.0,NY,0,1,3,2,1,0,1,...,1,1,0,0,1,1,599.0,590.0,0.0,-9.0
4,10122799,8.0,NY,2,1,2,2,1,0,1,...,1,1,0,0,3,1,724.0,717.0,1.0,-7.0


In [37]:
# While transforming the datasets, we also change encoding of vectors C, D and G to work with XG Boost classifier
# So we save the original train/test dataset with this changed encoding

datapath = "Test_data"

save_file(pp_test.data, 'test_data_first2_quotes_CDG_remapped.csv', datapath)


Writing file.  "Test_data\test_data_first2_quotes_CDG_remapped.csv"


In [13]:
df_train_q2_q3 = pd.read_csv('WIP_data/training_data_with_quotes_2_3.csv')
df_train_q2_q3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73152 entries, 0 to 73151
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_ID          73152 non-null  int64  
 1   shopping_pt          73152 non-null  int64  
 2   state                73152 non-null  object 
 3   A                    73152 non-null  int64  
 4   B                    73152 non-null  int64  
 5   C                    73152 non-null  int64  
 6   D                    73152 non-null  int64  
 7   E                    73152 non-null  int64  
 8   F                    73152 non-null  int64  
 9   G                    73152 non-null  int64  
 10  cost                 73152 non-null  int64  
 11  group_size_3         73152 non-null  int64  
 12  homeowner_3          73152 non-null  int64  
 13  car_age_3            73152 non-null  int64  
 14  car_value_3          73152 non-null  int64  
 15  risk_factor_3        73152 non-null 

In [15]:
pp23 = PreProcess()
pp23.dataset(df_train_q2_q3)

X, y = pp23.transform(quote_nos=[2,3])

X.shape, y.shape

((73152, 94), (73152, 7))

In [14]:
df_train_q2_q3.head()

,customer_ID,shopping_pt,state,A,B,C,D,E,F,G,...,D_2,D_3,E_2,E_3,F_2,F_3,G_2,G_3,cost_2,cost_3
0,10109793,9,CO,1,1,3,3,0,2,1,...,3,3,0,0,2,2,1,1,656,656
1,10002231,8,OH,0,0,1,3,0,0,3,...,3,3,0,0,0,0,3,3,557,564
2,10150024,8,OH,1,1,2,3,0,2,3,...,3,3,0,0,3,3,3,3,617,617
3,10003949,8,FL,1,1,2,2,1,2,3,...,2,2,1,1,2,2,3,3,675,675
4,10103809,10,FL,1,1,1,3,1,1,3,...,3,3,1,1,1,1,3,3,617,617


In [16]:
X.head()

,shopping_pt,car_age_3,age_oldest_3,age_youngest_3,duration_previous_3,cost_2,cost_3,changes_step_3,cost_diff_step_3,state_AR,...,F_2_3,F_3_1,F_3_2,F_3_3,G_2_1,G_2_2,G_2_3,G_3_1,G_3_2,G_3_3
0,1.054006,-0.897173,-0.000878,0.134046,-0.639141,0.459097,0.443265,-0.531869,-0.072856,0,...,0,0,1,0,0,0,0,0,0,0
1,0.501449,0.826906,-1.207892,-1.069244,0.858809,-1.622300,-1.564636,-0.531869,0.261214,0,...,0,0,0,0,0,1,0,0,1,0
2,0.501449,2.033761,-0.058355,-1.355742,-0.639141,-0.360847,-0.407910,-0.531869,-0.072856,0,...,1,0,0,1,0,1,0,0,1,0
3,0.501449,-0.207541,-0.690600,-0.610848,-0.639141,0.858557,0.857941,-0.531869,-0.072856,0,...,0,0,1,0,0,1,0,0,1,0
4,1.606563,0.137274,-1.035461,-0.897345,-0.425148,-0.360847,-0.407910,-0.531869,-0.072856,0,...,0,1,0,0,0,1,0,0,1,0


In [17]:
y.head()

,A,B,C,D,E,F,G
0,1,1,2,2,0,2,0
1,0,0,0,2,0,0,2
2,1,1,1,2,0,2,2
3,1,1,1,1,1,2,2
4,1,1,0,2,1,1,2


In [18]:
datapath2 = "Transformed_data"

save_file(X, 'pre_processed_training_data_with_quotes_2_3.csv', datapath2)
save_file(y, 'training_data_target_columns_quotes_2_3.csv', datapath2)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "Transformed_data\pre_processed_training_data_with_quotes_2_3.csv"
A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "Transformed_data\training_data_target_columns_quotes_2_3.csv"


In [11]:
# Let's also save the pre_processor object

save_file(pp23, 'pre_processor_quotes_2_3.pkl', datapath2)


Writing file.  "Transformed_data\pre_processor_quotes_2_3.pkl"
